In [1]:
# ================================================================
# RAG CHATBOT TƯ VẤN QUY CHẾ SINH VIÊN - KAGGLE NOTEBOOK
# Kế thừa từ hệ thống đánh giá LLM với behavior control
# Platform: Kaggle (GPU T4 x2)
# ================================================================

# ================================================================
# BƯỚC 1: CÀI ĐẶT THƯ VIỆN
# ================================================================
print("🔧 Đang cài đặt các thư viện cần thiết...")

!pip install -q langchain langchain-community langchain-huggingface chromadb gradio accelerate bitsandbytes transformers sentence-transformers

print("✅ Hoàn tất cài đặt thư viện!")

🔧 Đang cài đặt các thư viện cần thiết...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 573.2 kB/s eta 0:00:000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 83.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import gc
import torch
import re
from typing import Dict, List, Tuple
from datetime import datetime
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import gradio as gr

print("✅ Import thư viện thành công!")


2026-02-11 08:31:56.332735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770798716.722462      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770798716.838288      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770798717.755786      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770798717.755833      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770798717.755836      55 computation_placer.cc:177] computation placer alr

✅ Import thư viện thành công!


In [3]:
!cp -r /kaggle/input/chroma-db-for-hust-regulations/* /kaggle/working/

In [4]:
# ================================================================
# CẤU HÌNH ĐƯỜNG DẪN (Thay đổi theo môi trường của bạn)
# ================================================================
EMBEDDING_PATH = "/kaggle/input/alibaba/transformers/default/1/embedding"  # Thay đổi đường dẫn này
VECTOR_DB_PATH = "/kaggle/working/chroma_db"  # Thay đổi đường dẫn này
MODEL_ID = "wanduc0701/hust_chatbot_qwen2.5_finetuned"  # Model được fine-tune cho quy chế sinh viên

print(f"📁 Embedding Path: {EMBEDDING_PATH}")
print(f"📁 VectorDB Path: {VECTOR_DB_PATH}")
print(f"🤖 Model ID: {MODEL_ID}")

📁 Embedding Path: /kaggle/input/alibaba/transformers/default/1/embedding
📁 VectorDB Path: /kaggle/working/chroma_db
🤖 Model ID: wanduc0701/hust_chatbot_qwen2.5_finetuned


In [5]:
# ================================================================
# BƯỚC 2: THIẾT KẾ PROMPT TEMPLATE (CHUẨN FORMAT QWEN2.5)
# ================================================================

class StudentRegulationPrompts:
    @staticmethod
    def create_qa_template() -> str:
        template = """<|im_start|>system
Bạn là một trợ lý AI chuyên tư vấn về quy chế sinh viên của Đại học Bách Khoa Hà Nội.
NHIỆM VỤ CỐT LÕI: Chỉ trả lời câu hỏi của người dùng dựa trên thông tin được cung cấp trong phần CONTEXT.

QUY TẮC TUYỆT ĐỐI:
1.  **CHỈ DỰA VÀO CONTEXT**: Tuyệt đối không sử dụng kiến thức bên ngoài. Toàn bộ câu trả lời phải được rút ra từ Context được cung cấp.
2.  **TRÍCH DẪN NGUỒN**: Luôn bắt đầu câu trả lời bằng cách trích dẫn nguồn: "Theo [Mục/Điều...] trong [Tên văn bản], thì...".
3.  **XỬ LÝ KHI KHÔNG CÓ THÔNG TIN**: Nếu Context trống hoặc không chứa thông tin liên quan để trả lời câu hỏi, bạn BẮT BUỘC phải trả lời chính xác như sau: "Xin lỗi, tài liệu được cung cấp không đề cập đến vấn đề này."
4.  **KHÔNG SUY DIỄN**: Không được tự ý suy diễn, giả định hay bịa đặt thông tin không có trong Context.
<|im_end|>
<|im_start|>user
Dưới đây là các thông tin trích dẫn từ văn bản quy chế (Context):
---------------------
{context}
---------------------

Dựa vào context trên, hãy trả lời câu hỏi sau:
{question}
<|im_end|>
<|im_start|>assistant
"""
        return template

print("✅ Đã cập nhật Prompt chuẩn format Qwen2.5!")


✅ Đã cập nhật Prompt chuẩn format Qwen2.5!


In [6]:
# ================================================================
# BƯỚC 3: TẢI MODEL & TOKENIZER
# ================================================================
print("\n🚀 Đang tải LLM với quantization 4-bit...")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)
model.eval()

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # [CHỈNH SỬA] Tăng giới hạn token để tránh câu trả lời bị cắt ngang
    temperature=0.0,    # [CHỈNH SỬA] Đặt temperature=0.0 để câu trả lời nhất quán và bám sát context
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

print("✅ Khởi tạo LLM Pipeline thành công!")



🚀 Đang tải LLM với quantization 4-bit...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Khởi tạo LLM Pipeline thành công!


In [7]:
# ================================================================
# BƯỚC 4: KHỞI TẠO RAG SYSTEM (SỬA LỖI METADATA)
# ================================================================
print("\n🔍 Đang khởi tạo RAG System...")

# [CHỈNH SỬA] Tạo lớp Retriever an toàn để xử lý metadata bị thiếu
class SafeMetadataRetriever(BaseRetriever):
    """
    Một lớp retriever bao bọc (wrapper) để đảm bảo các tài liệu
    luôn có các trường metadata cần thiết trước khi được đưa vào prompt.
    """
    base_retriever: BaseRetriever

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Lấy tài liệu từ retriever gốc
        docs = self.base_retriever._get_relevant_documents(query, run_manager=run_manager)

        # Kiểm tra và "vá" metadata cho từng tài liệu
        for doc in docs:
            # Nếu thiếu 'source_filename', lấy từ 'source' hoặc đặt giá trị mặc định
            if "source_filename" not in doc.metadata:
                doc.metadata["source_filename"] = doc.metadata.get("source", "Không rõ nguồn")
            # Nếu thiếu 'article_title', đặt giá trị mặc định
            if "article_title" not in doc.metadata:
                doc.metadata["article_title"] = "Không có tiêu đề cụ thể"
        return docs

# 1. Load Embedding & VectorStore
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_PATH,
    model_kwargs={'device': 'cuda', 'trust_remote_code':True},
    encode_kwargs={'normalize_embeddings': True},
)

vectorstore = Chroma(
    persist_directory=VECTOR_DB_PATH,
    embedding_function=embeddings
)

# 2. Tạo retriever gốc
base_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 3. [CHỈNH SỬA] Bọc retriever gốc bằng SafeMetadataRetriever
retriever = SafeMetadataRetriever(base_retriever=base_retriever)
print("✅ Đã áp dụng SafeMetadataRetriever để xử lý lỗi dữ liệu!")

# 4. Định nghĩa Prompt
QA_PROMPT = PromptTemplate(
    template=StudentRegulationPrompts.create_qa_template(),
    input_variables=["context", "question"]
)

# 5. Document Prompt
document_prompt = PromptTemplate(
    input_variables=["page_content", "source_filename", "article_title"],
    template="""
    ---------------------
    📄 Tài liệu: {source_filename}
    🔖 Mục: {article_title}
    📝 Nội dung: {page_content}
    ---------------------
    """
)

# 6. KHỞI TẠO RETRIEVAL QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever, # Sử dụng retriever đã được "vá" lỗi
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_PROMPT,
        "document_prompt": document_prompt
    },
    verbose=False
)

print("✅ RAG System Siêu Tốc đã sẵn sàng!")


🔍 Đang khởi tạo RAG System...


/tmp/ipykernel_55/1954822006.py:37: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Đã áp dụng SafeMetadataRetriever để xử lý lỗi dữ liệu!
✅ RAG System Siêu Tốc đã sẵn sàng!


In [8]:
# ================================================================
# BƯỚC 5: XỬ LÝ LOGIC CHAT
# ================================================================

def chat_with_rag(message: str) -> Tuple[str, Dict]:
    try:
        # Hệ thống giờ đây sẽ không bị crash khi gặp metadata thiếu
        response = qa_chain.invoke({"query": message})

        answer = response.get("result", "Xin lỗi, hệ thống đang gặp sự cố và không thể tạo câu trả lời.").strip()
        source_docs = response.get("source_documents", [])
        sources = []
        if source_docs:
            for i, doc in enumerate(source_docs, 1):
                metadata = doc.metadata
                # Sử dụng .get() với giá trị mặc định để an toàn hơn
                full_path = metadata.get("source_filename", metadata.get("source", "Không rõ"))
                file_name = os.path.basename(full_path)
                article_title = metadata.get("article_title", "Nội dung liên quan")

                sources.append({
                    "STT": i,
                    "Tên văn bản": file_name,
                    "Chi tiết": article_title,
                    "Nội dung trích dẫn": " ".join(doc.page_content.split())[:300] + "..."
                })

        metadata_output = {
            "num_sources": len(source_docs),
            "sources": sources,
            "timestamp": datetime.now().strftime("%H:%M:%S %d/%m/%Y")
        }
        return answer, metadata_output

    except Exception as e:
        print(f"Error in chat_with_rag: {str(e)}")
        # Trả về một thông báo lỗi thân thiện hơn cho người dùng
        error_message = f"❌ Đã xảy ra lỗi hệ thống. Vui lòng thử lại sau.\nChi tiết: {str(e)}"
        return error_message, {"num_sources": 0, "sources": []}

def clear_conversation():
    try:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("Conversation cleared and memory freed.")
        return [], "Chưa có dữ liệu.", None
    except Exception as e:
        print(f"Error clearing conversation: {e}")
        return [], "Lỗi khi dọn dẹp.", None



In [ ]:
# ================================================================
# BƯỚC 6: GIAO DIỆN GRADIO
# ================================================================
print("\n🎨 Đang khởi tạo giao diện Gradio...")

custom_css = """
#chatbot { height: 500px; overflow-y: auto; }
#sources { height: 350px; overflow-y: auto; background-color: #f0f2f6; border-radius: 8px; padding: 10px; border: 1px solid #dee2e6; }
"""

with gr.Blocks(css=custom_css, title="HUST Chatbot", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 🎓 Trợ lý Quy chế Sinh viên HUST")

    with gr.Row():
        with gr.Column(scale=7):
            chatbot = gr.Chatbot(
                elem_id="chatbot",
                label="Hội thoại",
                height=550,
                avatar_images=(None, "🤖"),
                bubble_full_width=False,
                show_copy_button=True
            )
            with gr.Row():
                msg = gr.Textbox(
                    label="Nhập câu hỏi của bạn tại đây...",
                    placeholder="VD: Điều kiện để được xét học bổng loại A là gì?",
                    show_label=False,
                    lines=1,
                    max_lines=3,
                    scale=5,
                    autofocus=True
                )
                submit_btn = gr.Button("Gửi", variant="primary", scale=1, min_width=100)
            clear_btn = gr.Button("🗑️ Xóa hội thoại cũ", variant="secondary", size="sm")

        with gr.Column(scale=3):
            with gr.Accordion("💡 Hướng dẫn sử dụng", open=False):
                gr.Markdown(
                    """
                    ### Cách sử dụng chatbot:
                    1.  **Đặt câu hỏi**: Nhập câu hỏi về quy chế sinh viên vào ô bên trái.
                    2.  **Xem phản hồi**: Chatbot sẽ trả lời dựa trên văn bản quy chế.
                    3.  **Kiểm tra nguồn**: Xem các văn bản được sử dụng ở cột bên phải.
                    ### Ví dụ câu hỏi:
                    - Điều kiện để được cấp bằng tốt nghiệp là gì?
                    - Quy định về điểm danh trong giờ học ra sao?
                    - Sinh viên bị cảnh cáo học tập khi nào?
                    ### Lưu ý:
                    - Chatbot được thiết kế để trả lời dựa trên tài liệu. Nếu không tìm thấy, nó sẽ từ chối.
                    - Nên xóa hội thoại cũ sau mỗi phiên hỏi-đáp để giải phóng tài nguyên.
                    """
                )
            gr.Markdown("### 📊 Trạng thái tìm kiếm")
            stats_display = gr.Markdown(value="Chưa có dữ liệu.", label="Thống kê")
            gr.Markdown("### 📚 Tài liệu tham khảo")
            sources_display = gr.JSON(label="Chi tiết nguồn", elem_id="sources", height=400)

    def user_message(user_input, chat_history):
        if not user_input.strip():
            return None, chat_history
        if chat_history is None: chat_history = []
        return "", chat_history + [[user_input, None]]

    def bot_response(chat_history):
        if not chat_history or not chat_history[-1][0]:
            return chat_history, "Chưa có dữ liệu.", None
        user_input = chat_history[-1][0]
        bot_reply, metadata = chat_with_rag(user_input)
        chat_history[-1][1] = bot_reply
        stats_text = f"**Tìm thấy:** {metadata['num_sources']} đoạn văn bản liên quan"
        if metadata['num_sources'] > 0:
            stats_text += f" | 🕒 {metadata.get('timestamp','')}"
        return chat_history, stats_text, metadata["sources"]

    msg.submit(user_message, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, [chatbot, stats_display, sources_display]
    )
    submit_btn.click(user_message, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, [chatbot, stats_display, sources_display]
    )
    clear_btn.click(clear_conversation, None, [chatbot, stats_display, sources_display], queue=False)

print("\n🌐 Khởi động Gradio...")
demo.queue().launch(share=True, debug=True, server_name="0.0.0.0")


🎨 Đang khởi tạo giao diện Gradio...

🌐 Khởi động Gradio...


/tmp/ipykernel_55/1695057564.py:17: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipykernel_55/1695057564.py:17: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://ec00cd8bc22a7fe3bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
